In [ ]:
import torch_geometric.transforms as T
from nltk.parse.corenlp import CoreNLPParser,CoreNLPDependencyParser
from tqdm.auto import trange, tqdm
import pandas as pd
import ast
import itertools
from transformers import get_scheduler
import torch
import wandb
import evaluate
from itertools import cycle
import numpy as np
import random
import time

In [ ]:
import transformers
transformers.__version__

'4.18.0'

In [ ]:
import torch_geometric as pyg
pyg.__version__

'2.2.0'

In [ ]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# load processed files

In [ ]:
file = '../../data/CCAT50/processed/author_0,1_sent_2_train.csv'
df = pd.read_csv(file)
df['homo_edges'] = df['homo_edges'].apply(ast.literal_eval)
df['hetoro_edges'] = df['hetoro_edges'].apply(ast.literal_eval)
df['pos_seqs'] = df['pos_seqs'].apply(ast.literal_eval)

In [ ]:
file = '../../data/CCAT50/processed/author_0,1_sent_2_val.csv'
df_val = pd.read_csv(file)
df_val['homo_edges'] = df_val['homo_edges'].apply(ast.literal_eval)
df_val['hetoro_edges'] = df_val['hetoro_edges'].apply(ast.literal_eval)
df_val['pos_seqs'] = df_val['pos_seqs'].apply(ast.literal_eval)

In [ ]:
def freeze_model(model, freeze_bert):
    '''
    if freeze_bert == True, freeze all layer. 
    if freeze_bert is a positive integer, freeze the bottom {freeze_bert} attention layers
    negative integer should also work
    '''
    if freeze_bert==True:
        for param in model.parameters():
            param.requires_grad = False
    elif isinstance(freeze_bert, (int, np.int32, np.int64, torch.int32, torch.int64)):
        for param in model.embeddings.parameters():
            param.requires_grad = False  
        for layer in model.encoder.layer[:freeze_bert]: 
            for param in layer.parameters():
                param.requires_grad = False  
    return model

In [ ]:
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from transformers import AutoTokenizer
from transformers.models.bert.modeling_bert import BertModel
import torch

In [ ]:
checkpoint = '/scratch/data_jz17d/result/pos_mlm_corenlp/pos_mlm_8/checkpoint-155000/'
tokenizer = AutoTokenizer.from_pretrained(checkpoint, local_files_only=True)

In [ ]:
bert = BertModel.from_pretrained(checkpoint, local_files_only=True, add_pooling_layer = False)
bert = bert.eval()

Some weights of the model checkpoint at /scratch/data_jz17d/result/pos_mlm_corenlp/pos_mlm_8/checkpoint-155000/ were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
from dataclasses import dataclass
@dataclass
class myGNNoutput:
    loss: None
    logit: None
    emb: None

In [ ]:
# English specific denpendency relations: https://universaldependencies.org/en/dep/
s = '''nsubj 	csubj
↳nsubj:pass 	↳csubj:pass
↳nsubj:outer 	↳csubj:outer
obj 	ccomp 	xcomp
iobj
obl 	advcl 	advmod
↳obl:npmod 	↳advcl:relcl
↳obl:tmod
vocative 	aux 	mark
discourse 	↳aux:pass
expl 	cop
nummod 	acl 	amod
  	↳acl:relcl
appos 	  	det
  	  	↳det:predet
nmod 	  	 
↳nmod:npmod
↳nmod:tmod
↳nmod:poss
compound 	flat
↳compound:prt 	↳flat:foreign
fixed 	goeswith
conj 	cc
  	↳cc:preconj
list 	parataxis 	orphan
dislocated 		reparandum
root 	punct 	dep'''
all_relations = []
s = s.split('\n')
for line in s:
    if '↳' in line:
        continue
    line = line.split('\t')
    for r in line:
        if r.strip() == '':
            continue
        all_relations.append(r.split(':')[0].strip())
if 'root' in all_relations:
    all_relations.remove('root')
    all_relations.append('ROOT')
    all_relations.append('case')      # manually add relation not in list
    all_relations.append('discourse')    # manually add relation not in list
all_relations = sorted(all_relations)

In [ ]:
# relations found from the data
unique_relations = set()
for hetoro_edges in df['hetoro_edges']:
    for edge in hetoro_edges:
        unique_relations.add(edge.split(':')[0])

In [ ]:
# if there are any new relation, add it manually
for r in unique_relations:
    if r not in all_relations:
        print(r)

In [ ]:
relation2id = {all_relations[i]:i for i in range(len(all_relations))}

In [ ]:
def get_loader(df, limit=None, batch_size=32, shuffle=True, max_length=128):
    data_list = []
    if limit is not None:
        dfnew = df.sample(frac=1).reset_index(drop=True)[:limit]
    else:
        dfnew = df
    for i in range(len(dfnew)):
        curr = dfnew.iloc[i]
        data = Data()
        edge_type = curr['hetoro_edges']
        data.edge_type_ids = torch.tensor([relation2id[t.split(':')[0]] for t in edge_type])
        data.edge_index = torch.tensor(curr['homo_edges']).T
        if data.edge_index.shape[1] > max_length-1:
            print(f"data {i} too long length {data.edge_index.shape[1]}")
            continue
        tokens = tokenizer(' '.join(curr['pos_seqs']), padding=True, truncation=True, max_length=max_length, return_tensors='pt')
        data.x = bert(**tokens).last_hidden_state.squeeze(0).detach()
        data.y = torch.tensor([curr['author']])
        data_list.append(data)

    loader = DataLoader(data_list, batch_size=batch_size, shuffle=shuffle)
    return loader 

In [ ]:
def preprocess_author_ids(df):
    assert 'author' in df, 'no column named "author" found in df'
    
    max_id, min_id = df['author'].max(), df['author'].min()
    mapping = {i+min_id:i for i in range(max_id-min_id+1)}
    df['author'] = df['author'].map(mapping)
    
    return df

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GATConv, GATv2Conv, TransformerConv, PDNConv, global_mean_pool

GNNtype2layer = {'GATConv':GATConv, 'GATv2Conv':GATv2Conv, 'TransformerConv':TransformerConv, 'PDNConv':PDNConv}

class myHeteroGNN(torch.nn.Module):
    def __init__(self, num_layers, num_classes, num_dep_type, heads, hidden_dim, dep_emb_dim=32, add_self_loops=False, gnntype='GATConv'):
        super().__init__()
        self.num_layers = num_layers
        self.num_classes = num_classes
        self.pos_emb_dim = 32 # this is determined by POS Bert
        self.heads = heads
        self.hidden_dim = hidden_dim
        self.dep_emb_dim = dep_emb_dim
        
        self.GNNlayer = GNNtype2layer[gnntype]
        
        self.add_self_loops = add_self_loops
        self.dep_emb_layer = nn.Embedding(num_dep_type, self.dep_emb_dim)
        
        self.gnns = nn.ModuleList()
        self.gnns.append(self.GNNlayer(self.pos_emb_dim, self.hidden_dim, heads = self.heads, add_self_loops=self.add_self_loops, edge_dim=self.dep_emb_dim))
        for i in range(self.num_layers-1):
            self.gnns.append(self.GNNlayer(self.hidden_dim * self.heads, self.hidden_dim, heads = self.heads, edge_dim=self.dep_emb_dim))
        
        self.classifier = nn.Linear(self.hidden_dim * self.heads, self.num_classes)
        self.lossfn = nn.CrossEntropyLoss()
        
    def forward(self, x, edge_index, edge_type_ids, batch, y, ptr, readout='pool'):
        edge_attr = self.dep_emb_layer(edge_type_ids)
        for i in range(self.num_layers):
            x = self.gnns[i](x, edge_index, edge_attr=edge_attr)
            x = F.relu(x)
        
        if readout == 'pool':
            x = global_mean_pool(x, batch) 
        elif readout == 'cls':
            x = x[ptr[:-1],:]
        
        x = F.dropout(x, training=self.training)
        logit = self.classifier(x)
        loss = self.lossfn(logit, y)
        return myGNNoutput(loss=loss, logit=logit, emb=x)

# run

In [ ]:
epochs = 60
warmup_ratio = 0.15

valid_loader = get_loader(df_val)
num_valid_steps = len(valid_loader)

LIMIT = [None]
NUM_LAYERS = [1,2,3,4]
LR = [1e-3, 5e-4]
HEADS = [1,2,3]
READOUT = ['pool']
GNNTYPE = ['TransformerConv'] # 'GATConv', 'GATv2Conv', 
HIDDEN_DIM = [32]
DEP_EMB_DIM = [16,32,64]
ADD_SELF_LOOPS = [False]

ARGS = itertools.product(LIMIT, NUM_LAYERS, LR, HEADS, READOUT, GNNTYPE, HIDDEN_DIM, DEP_EMB_DIM, ADD_SELF_LOOPS)
num_runs = len(list(ARGS))
run_pbar = trange(num_runs, leave=False)

ARGS = itertools.product(LIMIT, NUM_LAYERS, LR, HEADS, READOUT, GNNTYPE, HIDDEN_DIM, DEP_EMB_DIM, ADD_SELF_LOOPS)
for i_run, args in enumerate(ARGS):
    limit, num_layers, lr, heads, readout, gnntype, hidden_dim, dep_emb_dim, add_self_loops = args
    
    seed = np.random.randint(100)
    set_seed(seed)
    
    train_loader = get_loader(df, limit = limit)
    num_training_steps = len(train_loader)
    
    model = myHeteroGNN(num_layers, 
                        num_classes=2, 
                        num_dep_type=len(all_relations), 
                        heads=heads,
                        hidden_dim=hidden_dim,
                        dep_emb_dim=dep_emb_dim, 
                        add_self_loops=add_self_loops,
                        gnntype=gnntype
                        )
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    scheduler = get_scheduler("linear",
                            optimizer=optimizer,
                            num_warmup_steps=int(warmup_ratio*epochs*num_training_steps),
                            num_training_steps=epochs*num_training_steps)
    
    wconfig = {}
    wconfig['seed'] = seed
    wconfig['limit'] = limit
    wconfig['num_layers'] = num_layers
    wconfig['lr'] = lr
    wconfig['heads'] = heads
    wconfig['readout'] = readout
    wconfig['GNNtype'] = gnntype
    wconfig['hidden_dim'] = hidden_dim
    wconfig['dep_emb_dim'] = dep_emb_dim
    wconfig['add_self_loops'] = add_self_loops
    
    run = wandb.init(project="hetero POS GNN", 
                     entity="fsu-dsc-cil", 
                     dir='/scratch/data_jz17d/wandb_tmp/', 
                     config=wconfig,
                     name=f'run_{i_run}',
                     reinit=True,
                     settings=wandb.Settings(start_method="thread")
                    )
    
    pbar = trange(epochs*num_training_steps, leave=False)
    for i_epoch in range(epochs):
        model.train()
        for data in train_loader:
            data.to(device)
            optimizer.zero_grad()
            output = model(data.x, data.edge_index, data.edge_type_ids, data.batch, data.y, data.ptr, readout=readout)
            output.loss.backward()
            optimizer.step()
            scheduler.step()
            pbar.update(1)

        model.eval()
        metric = evaluate.load('accuracy')
        for data in valid_loader:
            data.to(device)
            output = model(data.x, data.edge_index, data.edge_type_ids, data.batch, data.y, data.ptr, readout=readout)
            logit = output.logit
            metric.add_batch(predictions=logit.argmax(axis=-1).cpu().detach().numpy(), references=data.y.cpu().numpy())
        evaluation = metric.compute()
        evaluation.update({'global_step':pbar.n})
        wandb.log(evaluation, step=pbar.n)
    
    run.finish()
    run_pbar.update(1)

  0%|          | 0/72 [00:00<?, ?it/s]

data 511 too long length 134
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


wandb: Currently logged in as: cpuyyp (fsu-dsc-cil). Use `wandb login --relogin` to force relogin


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/2460 [00:00<?, ?it/s]

...

  0%|          | 0/2460 [00:00<?, ?it/s]

accuracy,▁▁▁▃▇▇▆▇▆▇▇▇▇▇▇▇▇█▇███████▇█▇▇▇▇▇▇▇▇████
global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
accuracy,0.81481
global_step,2460


data 511 too long length 134
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/2460 [00:00<?, ?it/s]

accuracy,▁▁▃▆▇▆█▇▇█▇▇▆█████████▇███▇▇▇▇▇▇▇▇▇▇█▇▇▇
global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
accuracy,0.78395
global_step,2460


## testing dataset size

In [ ]:
preset_epochs = 60

warmup_ratio = 0.15

valid_loader = get_loader(df_val)
num_valid_steps = len(valid_loader)

LIMIT = [1250, 1000, 750, 500, 250]
NUM_LAYERS = [3]
LR = [1e-3, 5e-4]
HEADS = [2]
READOUT = ['pool']
GNNTYPE = ['GATConv']
HIDDEN_DIM = [16]
DEP_EMB_DIM = [32]
ADD_SELF_LOOPS = [False, True]

ARGS = itertools.product(LIMIT, NUM_LAYERS, LR, HEADS, READOUT, GNNTYPE, HIDDEN_DIM, DEP_EMB_DIM, ADD_SELF_LOOPS)
num_runs = len(list(ARGS))
run_pbar = trange(num_runs, leave=False)

ARGS = itertools.product(LIMIT, NUM_LAYERS, LR, HEADS, READOUT, GNNTYPE, HIDDEN_DIM, DEP_EMB_DIM, ADD_SELF_LOOPS)
for i_run, args in enumerate(ARGS):
    limit, num_layers, lr, heads, readout, gnntype, hidden_dim, dep_emb_dim, add_self_loops = args
    
    epochs = 1250*60//limit
    seed = np.random.randint(100)
    set_seed(seed)
    
    train_loader = get_loader(df, limit = limit)
    num_training_steps = len(train_loader)
    
    model = myHeteroGNN(num_layers, 
                        num_classes=2, 
                        num_dep_type=len(all_relations), 
                        heads=heads,
                        hidden_dim=hidden_dim,
                        dep_emb_dim=dep_emb_dim, 
                        add_self_loops=add_self_loops,
                        gnntype=gnntype
                        )
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    scheduler = get_scheduler("linear",
                            optimizer=optimizer,
                            num_warmup_steps=int(warmup_ratio*epochs*num_training_steps),
                            num_training_steps=epochs*num_training_steps)
    
    wconfig = {}
    wconfig['seed'] = seed
    wconfig['limit'] = limit
    wconfig['num_layers'] = num_layers
    wconfig['lr'] = lr
    wconfig['heads'] = heads
    wconfig['readout'] = readout
    wconfig['GNNtype'] = gnntype
    wconfig['hidden_dim'] = hidden_dim
    wconfig['dep_emb_dim'] = dep_emb_dim
    wconfig['add_self_loops'] = add_self_loops
    
    run = wandb.init(project="hetero POS GNN (dataset size 2)", 
                     entity="fsu-dsc-cil", 
                     dir='/scratch/data_jz17d/wandb_tmp/', 
                     config=wconfig,
                     name=f'run_{i_run}',
                     reinit=True)
    
    pbar = trange(epochs*num_training_steps, leave=False)
    for i_epoch in range(epochs):
        model.train()
        for data in train_loader:
            data.to(device)
            optimizer.zero_grad()
            output = model(data.x, data.edge_index, data.edge_type_ids, data.batch, data.y, data.ptr, readout=readout)
            output.loss.backward()
            optimizer.step()
            scheduler.step()
            pbar.update(1)

        model.eval()
        metric = evaluate.load('accuracy')
        for data in valid_loader:
            data.to(device)
            output = model(data.x, data.edge_index, data.edge_type_ids, data.batch, data.y, data.ptr, readout=readout)
            logit = output.logit
            metric.add_batch(predictions=logit.argmax(axis=-1).cpu().detach().numpy(), references=data.y.cpu().numpy())
        evaluation = metric.compute()
        evaluation.update({'global_step':pbar.n})
        wandb.log(evaluation, step=pbar.n)
    
    run.finish()
    run_pbar.update(1)

  0%|          | 0/20 [00:00<?, ?it/s]

data 369 too long length 134
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/2400 [00:00<?, ?it/s]

accuracy,▁▁▁▁▁▅▇▄█▅▇▇▇▇▇█▇▇██▇███▇▇█▇▇██▇█▇█▇████
global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
accuracy,0.75926
global_step,2400


data 498 too long length 134
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


...

  0%|          | 0/2400 [00:00<?, ?it/s]

accuracy,▁▁▁▂▅▅▄▆█▇███████▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇
global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
accuracy,0.67284
global_step,2400


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/2400 [00:00<?, ?it/s]

accuracy,▁▁▃▄▄▄▅▆▆▇▇▇▇▇▇▇▇███████████████████████
global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
accuracy,0.74691
global_step,2400


## testing gnntype

In [ ]:
epochs = 60
warmup_ratio = 0.15

valid_loader = get_loader(df_val)
num_valid_steps = len(valid_loader)

LIMIT = [None]
NUM_LAYERS = [4]
LR = [1e-3, 5e-4]
HEADS = [2]
READOUT = ['pool']
GNNTYPE = ['GATConv', 'GATv2Conv', 'TransformerConv'] 
HIDDEN_DIM = [32]
DEP_EMB_DIM = [32]
ADD_SELF_LOOPS = [True, False]

ARGS = itertools.product(LIMIT, NUM_LAYERS, LR, HEADS, READOUT, GNNTYPE, HIDDEN_DIM, DEP_EMB_DIM, ADD_SELF_LOOPS)
num_runs = len(list(ARGS))
run_pbar = trange(num_runs, leave=False)

ARGS = itertools.product(LIMIT, NUM_LAYERS, LR, HEADS, READOUT, GNNTYPE, HIDDEN_DIM, DEP_EMB_DIM, ADD_SELF_LOOPS)
for i_run, args in enumerate(ARGS):
    limit, num_layers, lr, heads, readout, gnntype, hidden_dim, dep_emb_dim, add_self_loops = args
    
    seed = np.random.randint(100)
    set_seed(seed)
    
    train_loader = get_loader(df, limit = limit)
    num_training_steps = len(train_loader)
    
    model = myHeteroGNN(num_layers, 
                        num_classes=2, 
                        num_dep_type=len(all_relations), 
                        heads=heads,
                        hidden_dim=hidden_dim,
                        dep_emb_dim=dep_emb_dim, 
                        add_self_loops=add_self_loops,
                        gnntype=gnntype
                        )
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    scheduler = get_scheduler("linear",
                            optimizer=optimizer,
                            num_warmup_steps=int(warmup_ratio*epochs*num_training_steps),
                            num_training_steps=epochs*num_training_steps)
    
    wconfig = {}
    wconfig['seed'] = seed
    wconfig['limit'] = limit
    wconfig['num_layers'] = num_layers
    wconfig['lr'] = lr
    wconfig['heads'] = heads
    wconfig['readout'] = readout
    wconfig['GNNtype'] = gnntype
    wconfig['hidden_dim'] = hidden_dim
    wconfig['dep_emb_dim'] = dep_emb_dim
    wconfig['add_self_loops'] = add_self_loops
    
    run = wandb.init(project="hetero POS GNN (GNNtype)", 
                     entity="fsu-dsc-cil", 
                     dir='/scratch/data_jz17d/wandb_tmp/', 
                     config=wconfig,
                     name=f'run_{i_run}',
                     reinit=True,
                     settings=wandb.Settings(start_method="thread")
                    )
    
    pbar = trange(epochs*num_training_steps, leave=False)
    for i_epoch in range(epochs):
        model.train()
        for data in train_loader:
            data.to(device)
            optimizer.zero_grad()
            output = model(data.x, data.edge_index, data.edge_type_ids, data.batch, data.y, data.ptr, readout=readout)
            output.loss.backward()
            optimizer.step()
            scheduler.step()
            pbar.update(1)

        model.eval()
        metric = evaluate.load('accuracy')
        for data in valid_loader:
            data.to(device)
            output = model(data.x, data.edge_index, data.edge_type_ids, data.batch, data.y, data.ptr, readout=readout)
            logit = output.logit
            metric.add_batch(predictions=logit.argmax(axis=-1).cpu().detach().numpy(), references=data.y.cpu().numpy())
        evaluation = metric.compute()
        evaluation.update({'global_step':pbar.n})
        wandb.log(evaluation, step=pbar.n)
    
    run.finish()
    run_pbar.update(1)

  0%|          | 0/12 [00:00<?, ?it/s]

data 511 too long length 134
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/2460 [00:00<?, ?it/s]

accuracy,▁▁▁▁▄▂▆▇▆▆▇▇▇▆▇▆█▇▆▇▇██▆▆▇███▇▇▇▇▇▇▇█▇▇▇
global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
accuracy,0.75
global_step,2460


data 511 too long length 134
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


...

  0%|          | 0/2460 [00:00<?, ?it/s]

accuracy,▁▁▁▂▇▇▇▅▇▇█▇█████▇█████▇▇▇█▇▇█▇███▇▇████
global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
accuracy,0.78704
global_step,2460


data 511 too long length 134
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/2460 [00:00<?, ?it/s]

accuracy,▁▄▄▄▅▇▇▇▇▇▇▇▇▇███▇██████████████████████
global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
accuracy,0.80556
global_step,2460


## testing num of authors

In [ ]:
epochs = 60
warmup_ratio = 0.15
data_folder = '../../data/CCAT50/processed'

IDS = [[0,1], [2,3], [0,1,2], [3,4,5], [0,1,2,3], [4,5,6,7]]
NUM_SENTENCES = [2]
LIMIT = [None]
NUM_LAYERS = [4]
LR = [1e-3, 5e-4]
HEADS = [2]
READOUT = ['pool']
GNNTYPE = ['TransformerConv'] 
HIDDEN_DIM = [32]
DEP_EMB_DIM = [32]
ADD_SELF_LOOPS = [True]

ARGS = itertools.product(IDS, NUM_SENTENCES, LIMIT, NUM_LAYERS, LR, HEADS, READOUT, GNNTYPE, HIDDEN_DIM, DEP_EMB_DIM, ADD_SELF_LOOPS)
num_runs = len(list(ARGS))
run_pbar = trange(num_runs, leave=False)

ARGS = itertools.product(IDS, NUM_SENTENCES, LIMIT, NUM_LAYERS, LR, HEADS, READOUT, GNNTYPE, HIDDEN_DIM, DEP_EMB_DIM, ADD_SELF_LOOPS)
for i_run, args in enumerate(ARGS):
    ids, num_sentences, limit, num_layers, lr, heads, readout, gnntype, hidden_dim, dep_emb_dim, add_self_loops = args
    
    seed = np.random.randint(100)
    set_seed(seed)
    
    str_author = ','.join(map(str, ids))
    file = f"author_{str_author}_sent_{num_sentences}_train.csv"
    df = pd.read_csv(f'{data_folder}/{file}')
    df['homo_edges'] = df['homo_edges'].apply(ast.literal_eval)
    df['hetoro_edges'] = df['hetoro_edges'].apply(ast.literal_eval)
    df['pos_seqs'] = df['pos_seqs'].apply(ast.literal_eval)
    df['upos_seqs'] = df['upos_seqs'].apply(ast.literal_eval)
    df = preprocess_author_ids(df)
    train_loader = get_loader(df, limit = limit)
    num_training_steps = len(train_loader)
    
    file = f"author_{str_author}_sent_{num_sentences}_val.csv"
    df_val = pd.read_csv(f'{data_folder}/{file}')
    df_val['homo_edges'] = df_val['homo_edges'].apply(ast.literal_eval)
    df_val['hetoro_edges'] = df_val['hetoro_edges'].apply(ast.literal_eval)
    df_val['pos_seqs'] = df_val['pos_seqs'].apply(ast.literal_eval)
    df_val['upos_seqs'] = df_val['upos_seqs'].apply(ast.literal_eval)
    df_val = preprocess_author_ids(df_val)
    valid_loader = get_loader(df_val)
    num_valid_steps = len(valid_loader)
    
    model = myHeteroGNN(num_layers, 
                        num_classes=len(ids), 
                        num_dep_type=len(all_relations), 
                        heads=heads,
                        hidden_dim=hidden_dim,
                        dep_emb_dim=dep_emb_dim, 
                        add_self_loops=add_self_loops,
                        gnntype=gnntype
                        )
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    scheduler = get_scheduler("linear",
                            optimizer=optimizer,
                            num_warmup_steps=int(warmup_ratio*epochs*num_training_steps),
                            num_training_steps=epochs*num_training_steps)
    
    wconfig = {}
    wconfig['seed'] = seed
    wconfig['ids'] = ids
    wconfig['num_authors'] = len(ids)
    wconfig['num_sentences'] = num_sentences
    wconfig['limit'] = limit
    wconfig['num_layers'] = num_layers
    wconfig['lr'] = lr
    wconfig['heads'] = heads
    wconfig['readout'] = readout
    wconfig['GNNtype'] = gnntype
    wconfig['hidden_dim'] = hidden_dim
    wconfig['dep_emb_dim'] = dep_emb_dim
    wconfig['add_self_loops'] = add_self_loops
    
    run = wandb.init(project="hetero POS GNN (num_authors)", 
                     entity="fsu-dsc-cil", 
                     dir='/scratch/data_jz17d/wandb_tmp/', 
                     config=wconfig,
                     name=f'run_{i_run}',
                     reinit=True,
                     settings=wandb.Settings(start_method="thread")
                    )
    
    pbar = trange(epochs*num_training_steps, leave=False)
    for i_epoch in range(epochs):
        model.train()
        for data in train_loader:
            data.to(device)
            optimizer.zero_grad()
            output = model(data.x, data.edge_index, data.edge_type_ids, data.batch, data.y, data.ptr, readout=readout)
            output.loss.backward()
            optimizer.step()
            scheduler.step()
            pbar.update(1)

        model.eval()
        acc = evaluate.load('accuracy')
        f1 = evaluate.load('f1', average='macro')
        for data in valid_loader:
            data.to(device)
            output = model(data.x, data.edge_index, data.edge_type_ids, data.batch, data.y, data.ptr, readout=readout)
            logit = output.logit
            acc.add_batch(predictions=logit.argmax(axis=-1).cpu().detach().numpy(), references=data.y.cpu().numpy())
            f1.add_batch(predictions=logit.argmax(axis=-1).cpu().detach().numpy(), references=data.y.cpu().numpy())
        evaluation = acc.compute()
        evaluation.update(f1.compute(average='macro'))
        evaluation.update({'global_step':pbar.n})
        wandb.log(evaluation, step=pbar.n)
    
    run.finish()
    run_pbar.update(1)

  0%|          | 0/12 [00:00<?, ?it/s]

data 908 too long length 134
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/2460 [00:00<?, ?it/s]

accuracy,▁▂▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇██████▇███▇▇████
f1,▇▁▆▇▇█▇█▇█▇███▇▇█████▇██████████████████
global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
accuracy,0.75455
f1,0.73955
global_step,2460


data 908 too long length 134
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


...

  0%|          | 0/2460 [00:00<?, ?it/s]

accuracy,▁▁▂▄▄▆▇▇▇▇▆▆████▇████▇██████████████████
f1,▁▂▂▅▇████▇▇▇████████████████████████████
global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
accuracy,0.82143
f1,0.82085
global_step,2460


data 908 too long length 134
data 1469 too long length 146
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/3660 [00:00<?, ?it/s]

ValueError: Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted'].

In [ ]:
evaluate.load('f1', average='macro')

EvaluationModule(name: "f1", module_type: "metric", features: {'predictions': Value(dtype='int32', id=None), 'references': Value(dtype='int32', id=None)}, usage: """
Args:
    predictions (`list` of `int`): Predicted labels.
    references (`list` of `int`): Ground truth labels.
    labels (`list` of `int`): The set of labels to include when `average` is not set to `'binary'`, and the order of the labels if `average` is `None`. Labels present in the data can be excluded, for example to calculate a multiclass average ignoring a majority negative class. Labels not present in the data will result in 0 components in a macro average. For multilabel targets, labels are column indices. By default, all labels in `predictions` and `references` are used in sorted order. Defaults to None.
    pos_label (`int`): The class to be considered the positive class, in the case where `average` is set to `binary`. Defaults to 1.
    average (`string`): This parameter is required for multiclass/multilabel t